In [1]:
from IPython.core.debugger import set_trace

In [2]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
%config IPCompleter.greedy = True
import json
import csv

#filepath = "C:/Users/Sebastian/Desktop/Uni/_Bachelorarbeit/data/data.dsv"
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
ts = "COLLECTION_TIMESTAMP"

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  if __name__ == '__main__':


In [3]:
import pickle
with open("combined_pblm1.pkl", "rb") as pklfile:
    df = pickle.load(pklfile)

In [4]:
def get10and15MinuteMetrics(df):
    tens = [];
    fifteens = [];
    allMetrics = df.METRIC_COLUMN.unique()
    for metric in allMetrics:
        samples = df[df.METRIC_COLUMN == metric].iloc[:5]
        if(len(samples) <= 1):
            print("not enough data:", metric)
            continue
        if(is10MinuteInterval(samples[ts])):
            #print("10minute: ", metric)
            tens.append(metric)
        else:
            #print("15minute: ", metric)
            fifteens.append(metric)
    return tens, fifteens
        
        
def is10MinuteInterval(timestamps):
    timedifferences = 0
    timestamps.reset_index(drop=True, inplace=True)
    limit = len(timestamps) - 1
    if(limit == 0):
        print("Only One timestamp:", timestamps)
        return True
    
    for i, curr in enumerate(timestamps):
        if i >= limit:
            break
        #print("i", i, len(timestamps), curr, timestamps[i+1])
        difference  = (timestamps[i+1] - curr).total_seconds()
        timedifferences += (difference / 60)
    return abs(10 - timedifferences/limit) < abs(15 - timedifferences/limit)

In [5]:
tens, fifteens = get10and15MinuteMetrics(df)

In [6]:
i = 0
def createUnifiedTimestamps(row):
    global df
    global i
    import math
    ts = row.COLLECTION_TIMESTAMP
    newMinute = getClosestMinute(ts.minute, row.is10MinuteInterval)
#     print("newMin: ", newMinute, "10min?", is10MinuteInterval)
    newValue = math.nan #NaN initialisierung
    
    if(newMinute == 60): # ts.minute was e.g 59
        # newMinute will be set at end of method
        newMinute = 0;
        # add one Hour to the current timestamp
        deltaOneHour = pd.Timedelta(value=1, unit="hour")
        ts = ts + deltaOneHour
    
    if(row.is10MinuteInterval):        
        if(newMinute == 0):
            # take it as it is
            newValue = row.VALUE
        elif(newMinute == 10):
            #do nothing, covered by min=20, for clarity no "else:"
            pass
        elif(newMinute == 20):
            # get mean between 10, 20 --> 15
            newValue = getMeanBetweenTwoTimestamps(df, ts, metric = row.METRIC_COLUMN, firstValue = row.VALUE)
            newMinute = 15
        elif(newMinute == 30):
            #take it as it is
            newValue = row.VALUE
        elif(newMinute == 40):
            #do nothing, covered by min=40
            pass
        elif(newMinute == 50):
            # get mean between 40,50 --> 45
            newValue = getMeanBetweenTwoTimestamps(df, ts, metric = row.METRIC_COLUMN, firstValue = row.VALUE)
            newMinute = 45
        #print("replaced Value", row.VALUE)
        row.VALUE = newValue #set it to nan to drop it later on
        #print("with", row.VALUE)
        
    #print("replaced Timestamp", ts)
    ts = ts.replace(minute = newMinute, second = 0)
    #print("with Timestamp", ts)
    i = i+1
    if(i % 10000 == 0):
        print("Erreicht: ", i)
    row.unifiedTimestamp = ts # why does it not work without tihs now...
    return ts


def getClosestMinute(actualMinute, is10Minute = False):
    #needs to include 60, but if its 60, we need to add 1 hour in method above
    possibleValues = [0,15,30,45, 60]
    if is10Minute:
        possibleValues = [0, 10, 20, 30, 40, 50, 60] 
    currentClosest = possibleValues[0]
    for val in possibleValues:
        if abs(actualMinute - val) < abs(actualMinute - currentClosest):
            currentClosest = val
    return currentClosest

def getMeanBetweenTwoTimestamps(df, firstTimestamp, metric, firstValue):
    precedingMin = firstTimestamp.minute - 10 # get for 10 if min=20, and 40 if min=50
    precedingTimestamp  = firstTimestamp.replace(minute = precedingMin)
    matchingRow = df.loc[(df["unifiedTimestamp"] == precedingTimestamp) & (df["METRIC_COLUMN"] == metric)]
    mean = (matchingRow.VALUE + firstValue) / 2
    return mean
    

In [7]:
newcol = "is10MinuteInterval"
dropCols = ["TARGET_TYPE", "TARGET_GUID", "METRIC_NAME", "METRIC_TYPE", "METRIC_GUID", "METRIC_LABEL", "COLUMN_LABEL"]
df[newcol] = df.apply(lambda row: row.METRIC_COLUMN in tens, axis=1)

In [8]:
#IMPORTANT
df["unifiedTimestamp"] = 0 # initialize it so it does exist for searching a matchingRow in getMeanBetweenTwoTimestamps

In [9]:
df["unifiedTimestamp"] = df.apply(createUnifiedTimestamps, axis=1)

Erreicht:  10000
Erreicht:  20000
Erreicht:  30000
Erreicht:  40000
Erreicht:  50000
Erreicht:  60000
Erreicht:  70000
Erreicht:  80000
Erreicht:  90000
Erreicht:  100000
Erreicht:  110000
Erreicht:  120000
Erreicht:  130000
Erreicht:  140000
Erreicht:  150000
Erreicht:  160000
Erreicht:  170000
Erreicht:  180000
Erreicht:  190000
Erreicht:  200000
Erreicht:  210000
Erreicht:  220000
Erreicht:  230000
Erreicht:  240000
Erreicht:  250000
Erreicht:  260000
Erreicht:  270000
Erreicht:  280000
Erreicht:  290000
Erreicht:  300000
Erreicht:  310000
Erreicht:  320000
Erreicht:  330000
Erreicht:  340000
Erreicht:  350000
Erreicht:  360000
Erreicht:  370000
Erreicht:  380000
Erreicht:  390000
Erreicht:  400000
Erreicht:  410000
Erreicht:  420000
Erreicht:  430000
Erreicht:  440000
Erreicht:  450000
Erreicht:  460000
Erreicht:  470000
Erreicht:  480000
Erreicht:  490000
Erreicht:  500000
Erreicht:  510000
Erreicht:  520000
Erreicht:  530000
Erreicht:  540000
Erreicht:  550000
Erreicht:  560000
E

In [10]:
df["relevant"] = True

In [11]:
df.head()

,TARGET_NAME,TARGET_TYPE,TARGET_GUID,METRIC_NAME,METRIC_TYPE,METRIC_COLUMN,METRIC_GUID,METRIC_LABEL,COLUMN_LABEL,COLLECTION_TIMESTAMP,VALUE,is10MinuteInterval,unifiedTimestamp,relevant
41325908,pblm_pblm1,oracle_database,78FAAFBC6AF632D4E05311CAC5825D80,instance_throughput,0,enqwaits_ps,9D4B219FF919DE584FC952F84FDA14F2,Throughput,Enqueue Waits (per second),2019-12-25 08:40:27,",272",True,2019-12-25 08:40:00,True
24984775,pblm_pblm1,oracle_database,78FAAFBC6AF632D4E05311CAC5825D80,instance_throughput,0,usercalls_pt,D85DE7CA63144184CD53392815B5A189,Throughput,User Calls (per transaction),2019-12-25 08:40:27,"5,034",True,2019-12-25 08:40:00,True
25424559,pblm_pblm1,oracle_database,78FAAFBC6AF632D4E05311CAC5825D80,instance_throughput,0,recurscalls_ps,247A9AE0148523EA86D7E17874BBD563,Throughput,Recursive Calls (per second),2019-12-25 08:40:27,"372,97",True,2019-12-25 08:40:00,True
25841124,pblm_pblm1,oracle_database,78FAAFBC6AF632D4E05311CAC5825D80,instance_throughput,0,recurscalls_pt,3BF983EC8421A579590AED6E6F323437,Throughput,Recursive Calls (per transaction),2019-12-25 08:40:27,"1089,8",True,2019-12-25 08:40:00,True
26280908,pblm_pblm1,oracle_database,78FAAFBC6AF632D4E05311CAC5825D80,instance_throughput,0,logreads_ps,F3535548360FCE04402F0E8CD18CF028,Throughput,Session Logical Reads (per second),2019-12-25 08:40:27,"494,648",True,2019-12-25 08:40:00,True


In [12]:
with open("checkpointTs_4weeks.pkl", "wb") as pklfile:
    pickle.dump(df, pklfile)

In [13]:
df.head()

,TARGET_NAME,TARGET_TYPE,TARGET_GUID,METRIC_NAME,METRIC_TYPE,METRIC_COLUMN,METRIC_GUID,METRIC_LABEL,COLUMN_LABEL,COLLECTION_TIMESTAMP,VALUE,is10MinuteInterval,unifiedTimestamp,relevant
41325908,pblm_pblm1,oracle_database,78FAAFBC6AF632D4E05311CAC5825D80,instance_throughput,0,enqwaits_ps,9D4B219FF919DE584FC952F84FDA14F2,Throughput,Enqueue Waits (per second),2019-12-25 08:40:27,",272",True,2019-12-25 08:40:00,True
24984775,pblm_pblm1,oracle_database,78FAAFBC6AF632D4E05311CAC5825D80,instance_throughput,0,usercalls_pt,D85DE7CA63144184CD53392815B5A189,Throughput,User Calls (per transaction),2019-12-25 08:40:27,"5,034",True,2019-12-25 08:40:00,True
25424559,pblm_pblm1,oracle_database,78FAAFBC6AF632D4E05311CAC5825D80,instance_throughput,0,recurscalls_ps,247A9AE0148523EA86D7E17874BBD563,Throughput,Recursive Calls (per second),2019-12-25 08:40:27,"372,97",True,2019-12-25 08:40:00,True
25841124,pblm_pblm1,oracle_database,78FAAFBC6AF632D4E05311CAC5825D80,instance_throughput,0,recurscalls_pt,3BF983EC8421A579590AED6E6F323437,Throughput,Recursive Calls (per transaction),2019-12-25 08:40:27,"1089,8",True,2019-12-25 08:40:00,True
26280908,pblm_pblm1,oracle_database,78FAAFBC6AF632D4E05311CAC5825D80,instance_throughput,0,logreads_ps,F3535548360FCE04402F0E8CD18CF028,Throughput,Session Logical Reads (per second),2019-12-25 08:40:27,"494,648",True,2019-12-25 08:40:00,True


In [14]:
def checkForRelevance(row):
    ts = row.unifiedTimestamp
    min = ts.minute
    relevant = False
    if (min == 0 or min == 15 or min == 30 or min == 45):
        relevant = True
    row.relevant = relevant
    return relevant

In [15]:
df["relevant"] = df.apply(checkForRelevance, axis=1)

In [16]:
df.head()

,TARGET_NAME,TARGET_TYPE,TARGET_GUID,METRIC_NAME,METRIC_TYPE,METRIC_COLUMN,METRIC_GUID,METRIC_LABEL,COLUMN_LABEL,COLLECTION_TIMESTAMP,VALUE,is10MinuteInterval,unifiedTimestamp,relevant
41325908,pblm_pblm1,oracle_database,78FAAFBC6AF632D4E05311CAC5825D80,instance_throughput,0,enqwaits_ps,9D4B219FF919DE584FC952F84FDA14F2,Throughput,Enqueue Waits (per second),2019-12-25 08:40:27,",272",True,2019-12-25 08:40:00,False
24984775,pblm_pblm1,oracle_database,78FAAFBC6AF632D4E05311CAC5825D80,instance_throughput,0,usercalls_pt,D85DE7CA63144184CD53392815B5A189,Throughput,User Calls (per transaction),2019-12-25 08:40:27,"5,034",True,2019-12-25 08:40:00,False
25424559,pblm_pblm1,oracle_database,78FAAFBC6AF632D4E05311CAC5825D80,instance_throughput,0,recurscalls_ps,247A9AE0148523EA86D7E17874BBD563,Throughput,Recursive Calls (per second),2019-12-25 08:40:27,"372,97",True,2019-12-25 08:40:00,False
25841124,pblm_pblm1,oracle_database,78FAAFBC6AF632D4E05311CAC5825D80,instance_throughput,0,recurscalls_pt,3BF983EC8421A579590AED6E6F323437,Throughput,Recursive Calls (per transaction),2019-12-25 08:40:27,"1089,8",True,2019-12-25 08:40:00,False
26280908,pblm_pblm1,oracle_database,78FAAFBC6AF632D4E05311CAC5825D80,instance_throughput,0,logreads_ps,F3535548360FCE04402F0E8CD18CF028,Throughput,Session Logical Reads (per second),2019-12-25 08:40:27,"494,648",True,2019-12-25 08:40:00,False


In [17]:
df[df.unifiedTimestamp == "2019-09-12 14:10:00"]

,TARGET_NAME,TARGET_TYPE,TARGET_GUID,METRIC_NAME,METRIC_TYPE,METRIC_COLUMN,METRIC_GUID,METRIC_LABEL,COLUMN_LABEL,COLLECTION_TIMESTAMP,VALUE,is10MinuteInterval,unifiedTimestamp,relevant


In [18]:
len(df.unifiedTimestamp.unique())

4184

In [19]:
tmp = df.drop(df[df.relevant == False].index)

In [20]:
len(tmp.unifiedTimestamp.unique())

2789

In [21]:
with open("checkpointTs_4weeks.pkl", "wb") as pklfile:
    pickle.dump(tmp, pklfile)

In [22]:
df = tmp

In [23]:
df[["unifiedTimestamp", "COLLECTION_TIMESTAMP", newcol]]

,unifiedTimestamp,COLLECTION_TIMESTAMP,is10MinuteInterval
15256114,2019-12-25 08:45:00,2019-12-25 08:40:27,False
90345429,2019-12-25 08:45:00,2019-12-25 08:43:35,False
97612983,2019-12-25 08:45:00,2019-12-25 08:44:46,False
97906268,2019-12-25 08:45:00,2019-12-25 08:44:46,False
98199553,2019-12-25 08:45:00,2019-12-25 08:44:46,False
...,...,...,...
156146027,2020-01-22 09:15:00,2020-01-22 09:17:19,False
156151385,2020-01-22 09:15:00,2020-01-22 09:17:19,False
156151386,2020-01-22 09:15:00,2020-01-22 09:17:19,False
156152137,2020-01-23 00:00:00,2020-01-23 00:01:32,False


In [24]:
len(df["METRIC_COLUMN"].unique())

194

In [25]:
import pickle
with open("4week_dataframeIstatUnifiedTs.pkl", "wb") as pickleFile:
    pickle.dump(df, pickleFile)

with open("multivariatUnifiedTs.pkl", "rb") as pickleFile:
    df = pickle.load(pickleFile)

In [3]:
with open("4week_dataframeIstatUnifiedTs.pkl", "rb") as pickleFile:
    df = pickle.load(pickleFile)

In [4]:
df.head()

,TARGET_NAME,TARGET_TYPE,TARGET_GUID,METRIC_NAME,METRIC_TYPE,METRIC_COLUMN,METRIC_GUID,METRIC_LABEL,COLUMN_LABEL,COLLECTION_TIMESTAMP,VALUE,is10MinuteInterval,unifiedTimestamp,relevant
15256114,pblm_pblm1,oracle_database,78FAAFBC6AF632D4E05311CAC5825D80,instance_throughput,0,transactions_ps,AFAB5DA8631F9AA3309D30DA60969EC9,Throughput,Number of Transactions (per second),2019-12-25 08:40:27,",342",False,2019-12-25 08:45:00,True
90345429,pblm_pblm1,oracle_database,78FAAFBC6AF632D4E05311CAC5825D80,db_inst_opt_sga,0,opt_sga_size,38AB0A7A1353AA466F5D8A5C122729AC,Optimal SGA,Optimal SGA Size (MB),2019-12-25 08:43:35,10240,False,2019-12-25 08:45:00,True
97612983,pblm_pblm1,oracle_database,78FAAFBC6AF632D4E05311CAC5825D80,memory_usage_sga_pga,0,sga_total,0DD82DEF4D7B007FB0D5492E417A7C26,SGA and PGA usage,SGA Size(MB),2019-12-25 08:44:46,"20479,998",False,2019-12-25 08:45:00,True
97906268,pblm_pblm1,oracle_database,78FAAFBC6AF632D4E05311CAC5825D80,memory_usage_sga_pga,0,fixed_sga,55A67FB14E2002C469F5307F649EF70F,SGA and PGA usage,Fixed SGA(MB),2019-12-25 08:44:46,"21,732",False,2019-12-25 08:45:00,True
98199553,pblm_pblm1,oracle_database,78FAAFBC6AF632D4E05311CAC5825D80,memory_usage_sga_pga,0,java_pool,8B808C91B0DACF7532DF73319EBDD8B5,SGA and PGA usage,Java Pool(MB),2019-12-25 08:44:46,320,False,2019-12-25 08:45:00,True


In [7]:
from math import nan
allMetrics = df.METRIC_COLUMN.unique()
for metric in allMetrics:
    df[metric] = nan

In [9]:
df.columns

Index(['TARGET_NAME', 'TARGET_TYPE', 'TARGET_GUID', 'METRIC_NAME',
       'METRIC_TYPE', 'METRIC_COLUMN', 'METRIC_GUID', 'METRIC_LABEL',
       'COLUMN_LABEL', 'COLLECTION_TIMESTAMP',
       ...
       'FILE_SIZE', 'InstrumentationPresent', 'NeedToInstrument',
       'OCMConfigured', 'waitClassName', 'if_type', 'dumpDir', 'Session Count',
       'username', 'machine'],
      dtype='object', length=208)

In [11]:
df.head()

,TARGET_NAME,TARGET_TYPE,TARGET_GUID,METRIC_NAME,METRIC_TYPE,METRIC_COLUMN,METRIC_GUID,METRIC_LABEL,COLUMN_LABEL,COLLECTION_TIMESTAMP,VALUE,is10MinuteInterval,unifiedTimestamp,relevant,transactions_ps,opt_sga_size,sga_total,fixed_sga,java_pool,shared_pool,buffer_cache,pga_total,other_sga_memory,log_buffer,large_pool,streams_pool,total_pga_allocated,hardParseBindMismElTimePs,javaExecutionElapsedTimePs,plsqlEexecutionElapsedTimePs,connManageCallElapsedTimePs,failedParseElapsedTimePs,dbTimePs,hardParseSharCritElTimePs,plsqlCompilationElapsedTimePs,bgCpuTimePs,sqlExecuteElapsedTimePs,parseTimeElapsedPs,failedParseSharMemElTimePs,repeatedBindElapsedTimePs,dbCpuPs,rmanCpuTimeBackupRestorePs,inboundPlsqlRpcElapsedTimePs,sequenceLoadElapsedTimePs,hardParseElapsedTimePs,averageWaitTime,totalWaitsFG,averageWaitTimeFG,totalWaitTimeFG,totalWaits,totalWaitTime,shared_free_pct,large_free_pct,java_free_pct,session_usage,logons,user_limit,opencursors,process_usage,total_memory,genericErrors,archiveHungErrors,blockCorruptErrors,mediaFailureErrors,sessTerminateErrors,user_cpu_time_cnt,avg_user_cpu_time_pct,user_wait_time_pct,host_cpu_usage_pct,other_wait_cnt,userio_wait_cnt,avg_sync_singleblk_read_latency,sortsdisk_ps,physreads_pt,iombs_ps,recurscalls_pt,physreadslob_pt,logreads_pt,sortsdisk_pt,consistentreadgets_ps,redowrites_ps,recurscalls_ps,softparse_pct,failedparses_ps,userrollbackundorec_pt,physreadsdir_ps,sortsrows_ps,dbblkgets_ps,redosize_ps,dbtime_ps,logreads_ps,tabscanslong_ps,dbtime_pt,dbblkgets_pt,iorequests_ps,redosize_pt,executeswoparse_pct,networkbytes_ps,tabscanslong_pt,usercall_pct,rows_psort,executions_ps,usercalls_ps,physwriteslob_ps,logons_pt,opncurs_ps,physwritesdir_pt,avg_active_sessions,parses_ps,crblks_ps,dbblkchanges_ps,enqtimeouts_ps,leafnodesplits_pt,commits_ps,physwrites_pt,dbblkchanges_pt,branchnodesplits_pt,indxscanstotal_ps,parses_pt,physwrites_ps,sortsrows_pt,leafnodesplits_ps,tabscanstotal_pt,tabscanstotal_ps,sortsmemory_ps,enqreqs_pt,hardparses_ps,sortsmemory_pt,enqwaits_pt,crundorecs_ps,enqtimeouts_pt,physreadsdir_pt,crblks_pt,executions_pt,branchnodesplits_ps,enqreqs_ps,physreadslob_ps,physwriteslob_pt,consistentreadchanges_pt,commits_pt,userrollbackundorec_ps,failedparses_pt,crundorecs_pt,indxscansfull_ps,physreads_ps,indxscansfull_pt,consistentreadchanges_ps,hardparses_pt,bgcheckpoints_ps,logons_ps,dbwrcheckpoints_ps,usercalls_pt,consistentreadgets_pt,redowrites_pt,enqwaits_ps,indxscanstotal_pt,enqdeadlocks_pt,rollbacks_ps,physwritesdir_ps,rollbacks_pt,opncurs_pt,enqdeadlocks_ps,avg_users_waiting_on_class,dbtime_waitclass_pct,interconnect_rate,corrupt,currentgets_cs,lost,cr_request_cs,time_cs,libcache_miss_pct,dictionaryhit_pct,pxdwngrd75_ps,cursorcachehit_pct,redologalloc_hit_pct,cpuusage_pt,pxdwngrd25_ps,dictionarymiss_pct,response_time_pt,bufcachehit_pct,cpu_time_pct,pxdwngrdserial_ps,pgacachehit_pct,cpuusage_ps,pxdwngrdserial_pt,pxdwngrd50_ps,libcache_hit_pct,inmem_sort_pct,dumpAvail,dumpTotal,dumpUsedPercent,dumpUsed,scn_intrinsic_growth_rate,max_tot_cpu_usage_ps,avg_tot_cpu_usage_ps,XML_FILE_SIZE,BIN_FILE_SIZE,AUD_FILE_SIZE,FILE_SIZE,InstrumentationPresent,NeedToInstrument,OCMConfigured,waitClassName,if_type,dumpDir,Session Count,username,machine
15256114,pblm_pblm1,oracle_database,78FAAFBC6AF632D4E05311CAC5825D80,instance_throughput,0,transactions_ps,AFAB5DA8631F9AA3309D30DA60969EC9,Throughput,Number of Transactions (per second),2019-12-25 08:40:27,",342",False,2019-12-25 08:45:00,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [12]:
result = pd.DataFrame(columns = df.columns)

In [13]:
result["timestamp"] = df.unifiedTimestamp.unique()

In [14]:
import math
def setValueToMetriccolumn(row):
    global result
    metric = row.METRIC_COLUMN
    ts = row.unifiedTimestamp
    (result.loc[result.timestamp == ts, metric]) = row.VALUE
    return row

In [15]:
df = df.apply(setValueToMetriccolumn, axis=1)

In [16]:
result.head()

,TARGET_NAME,TARGET_TYPE,TARGET_GUID,METRIC_NAME,METRIC_TYPE,METRIC_COLUMN,METRIC_GUID,METRIC_LABEL,COLUMN_LABEL,COLLECTION_TIMESTAMP,VALUE,is10MinuteInterval,unifiedTimestamp,relevant,transactions_ps,opt_sga_size,sga_total,fixed_sga,java_pool,shared_pool,buffer_cache,pga_total,other_sga_memory,log_buffer,large_pool,streams_pool,total_pga_allocated,hardParseBindMismElTimePs,javaExecutionElapsedTimePs,plsqlEexecutionElapsedTimePs,connManageCallElapsedTimePs,failedParseElapsedTimePs,dbTimePs,hardParseSharCritElTimePs,plsqlCompilationElapsedTimePs,bgCpuTimePs,sqlExecuteElapsedTimePs,parseTimeElapsedPs,failedParseSharMemElTimePs,repeatedBindElapsedTimePs,dbCpuPs,rmanCpuTimeBackupRestorePs,inboundPlsqlRpcElapsedTimePs,sequenceLoadElapsedTimePs,hardParseElapsedTimePs,averageWaitTime,totalWaitsFG,averageWaitTimeFG,totalWaitTimeFG,totalWaits,totalWaitTime,shared_free_pct,large_free_pct,java_free_pct,session_usage,logons,user_limit,opencursors,process_usage,total_memory,genericErrors,archiveHungErrors,blockCorruptErrors,mediaFailureErrors,sessTerminateErrors,user_cpu_time_cnt,avg_user_cpu_time_pct,user_wait_time_pct,host_cpu_usage_pct,other_wait_cnt,userio_wait_cnt,avg_sync_singleblk_read_latency,sortsdisk_ps,physreads_pt,iombs_ps,recurscalls_pt,physreadslob_pt,logreads_pt,sortsdisk_pt,consistentreadgets_ps,redowrites_ps,recurscalls_ps,softparse_pct,failedparses_ps,userrollbackundorec_pt,physreadsdir_ps,sortsrows_ps,dbblkgets_ps,redosize_ps,dbtime_ps,logreads_ps,tabscanslong_ps,dbtime_pt,dbblkgets_pt,iorequests_ps,redosize_pt,executeswoparse_pct,networkbytes_ps,tabscanslong_pt,usercall_pct,rows_psort,executions_ps,usercalls_ps,physwriteslob_ps,logons_pt,opncurs_ps,physwritesdir_pt,avg_active_sessions,parses_ps,crblks_ps,dbblkchanges_ps,enqtimeouts_ps,leafnodesplits_pt,commits_ps,physwrites_pt,dbblkchanges_pt,branchnodesplits_pt,indxscanstotal_ps,parses_pt,physwrites_ps,sortsrows_pt,leafnodesplits_ps,tabscanstotal_pt,tabscanstotal_ps,sortsmemory_ps,enqreqs_pt,hardparses_ps,sortsmemory_pt,enqwaits_pt,crundorecs_ps,enqtimeouts_pt,physreadsdir_pt,crblks_pt,executions_pt,branchnodesplits_ps,enqreqs_ps,physreadslob_ps,physwriteslob_pt,consistentreadchanges_pt,commits_pt,userrollbackundorec_ps,failedparses_pt,crundorecs_pt,indxscansfull_ps,physreads_ps,indxscansfull_pt,consistentreadchanges_ps,hardparses_pt,bgcheckpoints_ps,logons_ps,dbwrcheckpoints_ps,usercalls_pt,consistentreadgets_pt,redowrites_pt,enqwaits_ps,indxscanstotal_pt,enqdeadlocks_pt,rollbacks_ps,physwritesdir_ps,rollbacks_pt,opncurs_pt,enqdeadlocks_ps,avg_users_waiting_on_class,dbtime_waitclass_pct,interconnect_rate,corrupt,currentgets_cs,lost,cr_request_cs,time_cs,libcache_miss_pct,dictionaryhit_pct,pxdwngrd75_ps,cursorcachehit_pct,redologalloc_hit_pct,cpuusage_pt,pxdwngrd25_ps,dictionarymiss_pct,response_time_pt,bufcachehit_pct,cpu_time_pct,pxdwngrdserial_ps,pgacachehit_pct,cpuusage_ps,pxdwngrdserial_pt,pxdwngrd50_ps,libcache_hit_pct,inmem_sort_pct,dumpAvail,dumpTotal,dumpUsedPercent,dumpUsed,scn_intrinsic_growth_rate,max_tot_cpu_usage_ps,avg_tot_cpu_usage_ps,XML_FILE_SIZE,BIN_FILE_SIZE,AUD_FILE_SIZE,FILE_SIZE,InstrumentationPresent,NeedToInstrument,OCMConfigured,waitClassName,if_type,dumpDir,Session Count,username,machine,timestamp
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1,586",10240,"20479,998","21,732",320,4928,14336,"1144,103",384,"42,266",320,128,"1147,1",0,0,",394",0,0,",534",",001",0,",063",",531",",009",0,0,",143",0,0,0,",008",",61",140,",49",18,6522,18,"38,85358711341759","17,665987014770508",100,"29,5634920634921",111,",00000258442014515968",131,"49,6666666666667","21545,57",0,0,0,0,0,",207",",37","64,26478099258024","21,16630608177691","1,245",",011",",025",0,"2,876","1,084","942,09",0,"1891,678",0,"3021,098","9,9","1816,551","98,648",",09",0,",023","18022,693","626,461","105006,01","137,1","3647,559",0,"71,102","324,892","52,29","54457,662","52,851","816,778",0,",137",10804144,"655,464","2,494",",449",",21","641,351",",485","1,371","309,042",",007

In [17]:
df[df.METRIC_COLUMN == "InstrumentationPresent"]

,TARGET_NAME,TARGET_TYPE,TARGET_GUID,METRIC_NAME,METRIC_TYPE,METRIC_COLUMN,METRIC_GUID,METRIC_LABEL,COLUMN_LABEL,COLLECTION_TIMESTAMP,VALUE,is10MinuteInterval,unifiedTimestamp,relevant,transactions_ps,opt_sga_size,sga_total,fixed_sga,java_pool,shared_pool,buffer_cache,pga_total,other_sga_memory,log_buffer,large_pool,streams_pool,total_pga_allocated,hardParseBindMismElTimePs,javaExecutionElapsedTimePs,plsqlEexecutionElapsedTimePs,connManageCallElapsedTimePs,failedParseElapsedTimePs,dbTimePs,hardParseSharCritElTimePs,plsqlCompilationElapsedTimePs,bgCpuTimePs,sqlExecuteElapsedTimePs,parseTimeElapsedPs,failedParseSharMemElTimePs,repeatedBindElapsedTimePs,dbCpuPs,rmanCpuTimeBackupRestorePs,inboundPlsqlRpcElapsedTimePs,sequenceLoadElapsedTimePs,hardParseElapsedTimePs,averageWaitTime,totalWaitsFG,averageWaitTimeFG,totalWaitTimeFG,totalWaits,totalWaitTime,shared_free_pct,large_free_pct,java_free_pct,session_usage,logons,user_limit,opencursors,process_usage,total_memory,genericErrors,archiveHungErrors,blockCorruptErrors,mediaFailureErrors,sessTerminateErrors,user_cpu_time_cnt,avg_user_cpu_time_pct,user_wait_time_pct,host_cpu_usage_pct,other_wait_cnt,userio_wait_cnt,avg_sync_singleblk_read_latency,sortsdisk_ps,physreads_pt,iombs_ps,recurscalls_pt,physreadslob_pt,logreads_pt,sortsdisk_pt,consistentreadgets_ps,redowrites_ps,recurscalls_ps,softparse_pct,failedparses_ps,userrollbackundorec_pt,physreadsdir_ps,sortsrows_ps,dbblkgets_ps,redosize_ps,dbtime_ps,logreads_ps,tabscanslong_ps,dbtime_pt,dbblkgets_pt,iorequests_ps,redosize_pt,executeswoparse_pct,networkbytes_ps,tabscanslong_pt,usercall_pct,rows_psort,executions_ps,usercalls_ps,physwriteslob_ps,logons_pt,opncurs_ps,physwritesdir_pt,avg_active_sessions,parses_ps,crblks_ps,dbblkchanges_ps,enqtimeouts_ps,leafnodesplits_pt,commits_ps,physwrites_pt,dbblkchanges_pt,branchnodesplits_pt,indxscanstotal_ps,parses_pt,physwrites_ps,sortsrows_pt,leafnodesplits_ps,tabscanstotal_pt,tabscanstotal_ps,sortsmemory_ps,enqreqs_pt,hardparses_ps,sortsmemory_pt,enqwaits_pt,crundorecs_ps,enqtimeouts_pt,physreadsdir_pt,crblks_pt,executions_pt,branchnodesplits_ps,enqreqs_ps,physreadslob_ps,physwriteslob_pt,consistentreadchanges_pt,commits_pt,userrollbackundorec_ps,failedparses_pt,crundorecs_pt,indxscansfull_ps,physreads_ps,indxscansfull_pt,consistentreadchanges_ps,hardparses_pt,bgcheckpoints_ps,logons_ps,dbwrcheckpoints_ps,usercalls_pt,consistentreadgets_pt,redowrites_pt,enqwaits_ps,indxscanstotal_pt,enqdeadlocks_pt,rollbacks_ps,physwritesdir_ps,rollbacks_pt,opncurs_pt,enqdeadlocks_ps,avg_users_waiting_on_class,dbtime_waitclass_pct,interconnect_rate,corrupt,currentgets_cs,lost,cr_request_cs,time_cs,libcache_miss_pct,dictionaryhit_pct,pxdwngrd75_ps,cursorcachehit_pct,redologalloc_hit_pct,cpuusage_pt,pxdwngrd25_ps,dictionarymiss_pct,response_time_pt,bufcachehit_pct,cpu_time_pct,pxdwngrdserial_ps,pgacachehit_pct,cpuusage_ps,pxdwngrdserial_pt,pxdwngrd50_ps,libcache_hit_pct,inmem_sort_pct,dumpAvail,dumpTotal,dumpUsedPercent,dumpUsed,scn_intrinsic_growth_rate,max_tot_cpu_usage_ps,avg_tot_cpu_usage_ps,XML_FILE_SIZE,BIN_FILE_SIZE,AUD_FILE_SIZE,FILE_SIZE,InstrumentationPresent,NeedToInstrument,OCMConfigured,waitClassName,if_type,dumpDir,Session Count,username,machine
10892914,pblm_pblm1,oracle_database,78FAAFBC6AF632D4E05311CAC5825D80,OCMInstrumentation,0,InstrumentationPresent,9CC05EF24BE1AC01F19D5195D2B6959A,OCM Instrumentation,Instrumentation Present,2019-12-26 04:03:50,1,False,2019-12-26 04:00:00,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [18]:
result = result.set_index("timestamp")

In [19]:
result = result.dropna(axis=1, how="all")

In [20]:
result["bgCpuTimePs"].isnull()

timestamp
2019-12-25 08:45:00    False
2019-12-25 09:00:00    False
2019-12-25 09:15:00    False
2019-12-25 09:30:00    False
2019-12-25 09:45:00    False
                       ...  
2020-01-23 08:45:00    False
2020-01-23 09:00:00    False
2020-01-23 09:15:00    False
2020-01-23 09:30:00    False
2020-01-23 09:45:00    True 
Name: bgCpuTimePs, Length: 2789, dtype: bool

In [21]:
with open("4week_checkpointTransformed.pkl", "wb") as pklfile:
    pickle.dump(result, pklfile)

In [22]:
result.head()

,transactions_ps,opt_sga_size,sga_total,fixed_sga,java_pool,shared_pool,buffer_cache,pga_total,other_sga_memory,log_buffer,large_pool,streams_pool,total_pga_allocated,hardParseBindMismElTimePs,javaExecutionElapsedTimePs,plsqlEexecutionElapsedTimePs,connManageCallElapsedTimePs,failedParseElapsedTimePs,dbTimePs,hardParseSharCritElTimePs,plsqlCompilationElapsedTimePs,bgCpuTimePs,sqlExecuteElapsedTimePs,parseTimeElapsedPs,failedParseSharMemElTimePs,repeatedBindElapsedTimePs,dbCpuPs,rmanCpuTimeBackupRestorePs,inboundPlsqlRpcElapsedTimePs,sequenceLoadElapsedTimePs,hardParseElapsedTimePs,averageWaitTime,totalWaitsFG,averageWaitTimeFG,totalWaitTimeFG,totalWaits,totalWaitTime,shared_free_pct,large_free_pct,java_free_pct,session_usage,logons,user_limit,opencursors,process_usage,total_memory,genericErrors,archiveHungErrors,blockCorruptErrors,mediaFailureErrors,sessTerminateErrors,user_cpu_time_cnt,avg_user_cpu_time_pct,user_wait_time_pct,host_cpu_usage_pct,other_wait_cnt,userio_wait_cnt,avg_sync_singleblk_read_latency,sortsdisk_ps,physreads_pt,iombs_ps,recurscalls_pt,physreadslob_pt,logreads_pt,sortsdisk_pt,consistentreadgets_ps,redowrites_ps,recurscalls_ps,softparse_pct,failedparses_ps,userrollbackundorec_pt,physreadsdir_ps,sortsrows_ps,dbblkgets_ps,redosize_ps,dbtime_ps,logreads_ps,tabscanslong_ps,dbtime_pt,dbblkgets_pt,iorequests_ps,redosize_pt,executeswoparse_pct,networkbytes_ps,tabscanslong_pt,usercall_pct,rows_psort,executions_ps,usercalls_ps,physwriteslob_ps,logons_pt,opncurs_ps,physwritesdir_pt,avg_active_sessions,parses_ps,crblks_ps,dbblkchanges_ps,enqtimeouts_ps,leafnodesplits_pt,commits_ps,physwrites_pt,dbblkchanges_pt,branchnodesplits_pt,indxscanstotal_ps,parses_pt,physwrites_ps,sortsrows_pt,leafnodesplits_ps,tabscanstotal_pt,tabscanstotal_ps,sortsmemory_ps,enqreqs_pt,hardparses_ps,sortsmemory_pt,enqwaits_pt,crundorecs_ps,enqtimeouts_pt,physreadsdir_pt,crblks_pt,executions_pt,branchnodesplits_ps,enqreqs_ps,physreadslob_ps,physwriteslob_pt,consistentreadchanges_pt,commits_pt,userrollbackundorec_ps,failedparses_pt,crundorecs_pt,indxscansfull_ps,physreads_ps,indxscansfull_pt,consistentreadchanges_ps,hardparses_pt,bgcheckpoints_ps,logons_ps,dbwrcheckpoints_ps,usercalls_pt,consistentreadgets_pt,redowrites_pt,enqwaits_ps,indxscanstotal_pt,enqdeadlocks_pt,rollbacks_ps,physwritesdir_ps,rollbacks_pt,opncurs_pt,enqdeadlocks_ps,avg_users_waiting_on_class,dbtime_waitclass_pct,interconnect_rate,corrupt,currentgets_cs,lost,cr_request_cs,time_cs,libcache_miss_pct,dictionaryhit_pct,pxdwngrd75_ps,cursorcachehit_pct,redologalloc_hit_pct,cpuusage_pt,pxdwngrd25_ps,dictionarymiss_pct,response_time_pt,bufcachehit_pct,cpu_time_pct,pxdwngrdserial_ps,pgacachehit_pct,cpuusage_ps,pxdwngrdserial_pt,pxdwngrd50_ps,libcache_hit_pct,inmem_sort_pct,dumpAvail,dumpTotal,dumpUsedPercent,dumpUsed,scn_intrinsic_growth_rate,max_tot_cpu_usage_ps,avg_tot_cpu_usage_ps,XML_FILE_SIZE,BIN_FILE_SIZE,AUD_FILE_SIZE,FILE_SIZE,InstrumentationPresent,NeedToInstrument,OCMConfigured,waitClassName,if_type,dumpDir,Session Count,username,machine
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-12-25 08:45:00,"1,586",10240,"20479,998","21,732",320,4928,14336,"1144,103",384,"42,266",320,128,"1147,1",0,0,",394",0,0,",534",",001",0,",063",",531",",009",0,0,",143",0,0,0,",008",",61",140,",49",18,6522,18,"38,85358711341759","17,665987014770508",100,"29,5634920634921",111,",00000258442014515968",131,"49,6666666666667","21545,57",0,0,0,0,0,",207",",37","64,26478099258024","21,16630608177691","1,245",",011",",025",0,"2,876","1,084","942,09",0,"1891,678",0,"3021,098","9,9","1816,551","98,648",",09",0,",023","18022,693","626,461","105006,01","137,1","3647,559",0,"71,102","324,892","52,29","54457,662","52,851","816,778",0,",137",10804144,"655,464","2,494",",449",",21","641,351",",485","1,371","309,042",",007","604,81",0,",262","1,927",

In [23]:
result.columns.tolist()

['transactions_ps',
 'opt_sga_size',
 'sga_total',
 'fixed_sga',
 'java_pool',
 'shared_pool',
 'buffer_cache',
 'pga_total',
 'other_sga_memory',
 'log_buffer',
 'large_pool',
 'streams_pool',
 'total_pga_allocated',
 'hardParseBindMismElTimePs',
 'javaExecutionElapsedTimePs',
 'plsqlEexecutionElapsedTimePs',
 'connManageCallElapsedTimePs',
 'failedParseElapsedTimePs',
 'dbTimePs',
 'hardParseSharCritElTimePs',
 'plsqlCompilationElapsedTimePs',
 'bgCpuTimePs',
 'sqlExecuteElapsedTimePs',
 'parseTimeElapsedPs',
 'failedParseSharMemElTimePs',
 'repeatedBindElapsedTimePs',
 'dbCpuPs',
 'rmanCpuTimeBackupRestorePs',
 'inboundPlsqlRpcElapsedTimePs',
 'sequenceLoadElapsedTimePs',
 'hardParseElapsedTimePs',
 'averageWaitTime',
 'totalWaitsFG',
 'averageWaitTimeFG',
 'totalWaitTimeFG',
 'totalWaits',
 'totalWaitTime',
 'shared_free_pct',
 'large_free_pct',
 'java_free_pct',
 'session_usage',
 'logons',
 'user_limit',
 'opencursors',
 'process_usage',
 'total_memory',
 'genericErrors',
 'archi

In [24]:
irrelevantCols = ['scn_intrinsic_growth_rate',
'dumpTotal',
 'dumpUsed',
 'dumpUsedPercent',
 'dumpAvail',
 'CRITICAL_INCIDENTS',
 'WARNING_INCIDENTS',
 'InstrumentationPresent',
 'OCMConfigured',
 'NeedToInstrument',
 'FILE_SIZE',
 'AUD_FILE_SIZE',
 'BIN_FILE_SIZE',
 'XML_FILE_SIZE',
 'genericErrors',
 'archiveHungErrors',
 'blockCorruptErrors',
 'sessTerminateErrors',
 'mediaFailureErrors',
 'timeLine',
 'genericErrStack',
 'traceFileName',
 'waitClassName',
 'if_type',
 'dumpDir',
 'SEVERITY_INDEX',
 'alertLogName']

In [25]:
result.drop(columns= irrelevantCols, inplace=True )

KeyError: "['CRITICAL_INCIDENTS' 'WARNING_INCIDENTS' 'timeLine' 'genericErrStack'\n 'traceFileName' 'SEVERITY_INDEX' 'alertLogName'] not found in axis"

In [26]:
result.head()

,transactions_ps,opt_sga_size,sga_total,fixed_sga,java_pool,shared_pool,buffer_cache,pga_total,other_sga_memory,log_buffer,large_pool,streams_pool,total_pga_allocated,hardParseBindMismElTimePs,javaExecutionElapsedTimePs,plsqlEexecutionElapsedTimePs,connManageCallElapsedTimePs,failedParseElapsedTimePs,dbTimePs,hardParseSharCritElTimePs,plsqlCompilationElapsedTimePs,bgCpuTimePs,sqlExecuteElapsedTimePs,parseTimeElapsedPs,failedParseSharMemElTimePs,repeatedBindElapsedTimePs,dbCpuPs,rmanCpuTimeBackupRestorePs,inboundPlsqlRpcElapsedTimePs,sequenceLoadElapsedTimePs,hardParseElapsedTimePs,averageWaitTime,totalWaitsFG,averageWaitTimeFG,totalWaitTimeFG,totalWaits,totalWaitTime,shared_free_pct,large_free_pct,java_free_pct,session_usage,logons,user_limit,opencursors,process_usage,total_memory,genericErrors,archiveHungErrors,blockCorruptErrors,mediaFailureErrors,sessTerminateErrors,user_cpu_time_cnt,avg_user_cpu_time_pct,user_wait_time_pct,host_cpu_usage_pct,other_wait_cnt,userio_wait_cnt,avg_sync_singleblk_read_latency,sortsdisk_ps,physreads_pt,iombs_ps,recurscalls_pt,physreadslob_pt,logreads_pt,sortsdisk_pt,consistentreadgets_ps,redowrites_ps,recurscalls_ps,softparse_pct,failedparses_ps,userrollbackundorec_pt,physreadsdir_ps,sortsrows_ps,dbblkgets_ps,redosize_ps,dbtime_ps,logreads_ps,tabscanslong_ps,dbtime_pt,dbblkgets_pt,iorequests_ps,redosize_pt,executeswoparse_pct,networkbytes_ps,tabscanslong_pt,usercall_pct,rows_psort,executions_ps,usercalls_ps,physwriteslob_ps,logons_pt,opncurs_ps,physwritesdir_pt,avg_active_sessions,parses_ps,crblks_ps,dbblkchanges_ps,enqtimeouts_ps,leafnodesplits_pt,commits_ps,physwrites_pt,dbblkchanges_pt,branchnodesplits_pt,indxscanstotal_ps,parses_pt,physwrites_ps,sortsrows_pt,leafnodesplits_ps,tabscanstotal_pt,tabscanstotal_ps,sortsmemory_ps,enqreqs_pt,hardparses_ps,sortsmemory_pt,enqwaits_pt,crundorecs_ps,enqtimeouts_pt,physreadsdir_pt,crblks_pt,executions_pt,branchnodesplits_ps,enqreqs_ps,physreadslob_ps,physwriteslob_pt,consistentreadchanges_pt,commits_pt,userrollbackundorec_ps,failedparses_pt,crundorecs_pt,indxscansfull_ps,physreads_ps,indxscansfull_pt,consistentreadchanges_ps,hardparses_pt,bgcheckpoints_ps,logons_ps,dbwrcheckpoints_ps,usercalls_pt,consistentreadgets_pt,redowrites_pt,enqwaits_ps,indxscanstotal_pt,enqdeadlocks_pt,rollbacks_ps,physwritesdir_ps,rollbacks_pt,opncurs_pt,enqdeadlocks_ps,avg_users_waiting_on_class,dbtime_waitclass_pct,interconnect_rate,corrupt,currentgets_cs,lost,cr_request_cs,time_cs,libcache_miss_pct,dictionaryhit_pct,pxdwngrd75_ps,cursorcachehit_pct,redologalloc_hit_pct,cpuusage_pt,pxdwngrd25_ps,dictionarymiss_pct,response_time_pt,bufcachehit_pct,cpu_time_pct,pxdwngrdserial_ps,pgacachehit_pct,cpuusage_ps,pxdwngrdserial_pt,pxdwngrd50_ps,libcache_hit_pct,inmem_sort_pct,dumpAvail,dumpTotal,dumpUsedPercent,dumpUsed,scn_intrinsic_growth_rate,max_tot_cpu_usage_ps,avg_tot_cpu_usage_ps,XML_FILE_SIZE,BIN_FILE_SIZE,AUD_FILE_SIZE,FILE_SIZE,InstrumentationPresent,NeedToInstrument,OCMConfigured,waitClassName,if_type,dumpDir,Session Count,username,machine
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-12-25 08:45:00,"1,586",10240,"20479,998","21,732",320,4928,14336,"1144,103",384,"42,266",320,128,"1147,1",0,0,",394",0,0,",534",",001",0,",063",",531",",009",0,0,",143",0,0,0,",008",",61",140,",49",18,6522,18,"38,85358711341759","17,665987014770508",100,"29,5634920634921",111,",00000258442014515968",131,"49,6666666666667","21545,57",0,0,0,0,0,",207",",37","64,26478099258024","21,16630608177691","1,245",",011",",025",0,"2,876","1,084","942,09",0,"1891,678",0,"3021,098","9,9","1816,551","98,648",",09",0,",023","18022,693","626,461","105006,01","137,1","3647,559",0,"71,102","324,892","52,29","54457,662","52,851","816,778",0,",137",10804144,"655,464","2,494",",449",",21","641,351",",485","1,371","309,042",",007","604,81",0,",262","1,927",

In [27]:
result.drop(columns = ['InstrumentationPresent',
 'OCMConfigured',
 'NeedToInstrument', "waitClassName", "if_type", "dumpDir", "Session Count", "username", "machine",
 'FILE_SIZE',
 'AUD_FILE_SIZE',
 'BIN_FILE_SIZE',
 'XML_FILE_SIZE',], inplace = True)

In [28]:
result.head()

,transactions_ps,opt_sga_size,sga_total,fixed_sga,java_pool,shared_pool,buffer_cache,pga_total,other_sga_memory,log_buffer,large_pool,streams_pool,total_pga_allocated,hardParseBindMismElTimePs,javaExecutionElapsedTimePs,plsqlEexecutionElapsedTimePs,connManageCallElapsedTimePs,failedParseElapsedTimePs,dbTimePs,hardParseSharCritElTimePs,plsqlCompilationElapsedTimePs,bgCpuTimePs,sqlExecuteElapsedTimePs,parseTimeElapsedPs,failedParseSharMemElTimePs,repeatedBindElapsedTimePs,dbCpuPs,rmanCpuTimeBackupRestorePs,inboundPlsqlRpcElapsedTimePs,sequenceLoadElapsedTimePs,hardParseElapsedTimePs,averageWaitTime,totalWaitsFG,averageWaitTimeFG,totalWaitTimeFG,totalWaits,totalWaitTime,shared_free_pct,large_free_pct,java_free_pct,session_usage,logons,user_limit,opencursors,process_usage,total_memory,genericErrors,archiveHungErrors,blockCorruptErrors,mediaFailureErrors,sessTerminateErrors,user_cpu_time_cnt,avg_user_cpu_time_pct,user_wait_time_pct,host_cpu_usage_pct,other_wait_cnt,userio_wait_cnt,avg_sync_singleblk_read_latency,sortsdisk_ps,physreads_pt,iombs_ps,recurscalls_pt,physreadslob_pt,logreads_pt,sortsdisk_pt,consistentreadgets_ps,redowrites_ps,recurscalls_ps,softparse_pct,failedparses_ps,userrollbackundorec_pt,physreadsdir_ps,sortsrows_ps,dbblkgets_ps,redosize_ps,dbtime_ps,logreads_ps,tabscanslong_ps,dbtime_pt,dbblkgets_pt,iorequests_ps,redosize_pt,executeswoparse_pct,networkbytes_ps,tabscanslong_pt,usercall_pct,rows_psort,executions_ps,usercalls_ps,physwriteslob_ps,logons_pt,opncurs_ps,physwritesdir_pt,avg_active_sessions,parses_ps,crblks_ps,dbblkchanges_ps,enqtimeouts_ps,leafnodesplits_pt,commits_ps,physwrites_pt,dbblkchanges_pt,branchnodesplits_pt,indxscanstotal_ps,parses_pt,physwrites_ps,sortsrows_pt,leafnodesplits_ps,tabscanstotal_pt,tabscanstotal_ps,sortsmemory_ps,enqreqs_pt,hardparses_ps,sortsmemory_pt,enqwaits_pt,crundorecs_ps,enqtimeouts_pt,physreadsdir_pt,crblks_pt,executions_pt,branchnodesplits_ps,enqreqs_ps,physreadslob_ps,physwriteslob_pt,consistentreadchanges_pt,commits_pt,userrollbackundorec_ps,failedparses_pt,crundorecs_pt,indxscansfull_ps,physreads_ps,indxscansfull_pt,consistentreadchanges_ps,hardparses_pt,bgcheckpoints_ps,logons_ps,dbwrcheckpoints_ps,usercalls_pt,consistentreadgets_pt,redowrites_pt,enqwaits_ps,indxscanstotal_pt,enqdeadlocks_pt,rollbacks_ps,physwritesdir_ps,rollbacks_pt,opncurs_pt,enqdeadlocks_ps,avg_users_waiting_on_class,dbtime_waitclass_pct,interconnect_rate,corrupt,currentgets_cs,lost,cr_request_cs,time_cs,libcache_miss_pct,dictionaryhit_pct,pxdwngrd75_ps,cursorcachehit_pct,redologalloc_hit_pct,cpuusage_pt,pxdwngrd25_ps,dictionarymiss_pct,response_time_pt,bufcachehit_pct,cpu_time_pct,pxdwngrdserial_ps,pgacachehit_pct,cpuusage_ps,pxdwngrdserial_pt,pxdwngrd50_ps,libcache_hit_pct,inmem_sort_pct,dumpAvail,dumpTotal,dumpUsedPercent,dumpUsed,scn_intrinsic_growth_rate,max_tot_cpu_usage_ps,avg_tot_cpu_usage_ps
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-12-25 08:45:00,"1,586",10240,"20479,998","21,732",320,4928,14336,"1144,103",384,"42,266",320,128,"1147,1",0,0,",394",0,0,",534",",001",0,",063",",531",",009",0,0,",143",0,0,0,",008",",61",140,",49",18,6522,18,"38,85358711341759","17,665987014770508",100,"29,5634920634921",111,",00000258442014515968",131,"49,6666666666667","21545,57",0,0,0,0,0,",207",",37","64,26478099258024","21,16630608177691","1,245",",011",",025",0,"2,876","1,084","942,09",0,"1891,678",0,"3021,098","9,9","1816,551","98,648",",09",0,",023","18022,693","626,461","105006,01","137,1","3647,559",0,"71,102","324,892","52,29","54457,662","52,851","816,778",0,",137",10804144,"655,464","2,494",",449",",21","641,351",",485","1,371","309,042",",007","604,81",0,",262","1,927","23,285","313,663",",002","257,805","160,274","44,898","9346,834",",506",",084",",162","14,275","28,253","4,179","7,403",",374",",002",0,",012",",003","339,933",",003","54,477",0,"

In [32]:
result[~result.avg_tot_cpu_usage_ps.isnull()].avg_tot_cpu_usage_ps

timestamp
2019-12-25 09:30:00    8,88793132698182 
2019-12-25 10:30:00    3,408784609345012
2019-12-25 11:30:00    3,381302632626707
2019-12-25 12:30:00    3,386462245544893
2019-12-25 13:30:00    3,310598521370813
                             ...        
2020-01-23 05:30:00    3,574036613120352
2020-01-23 06:30:00    3,498166117908631
2020-01-23 07:30:00    3,484568689267122
2020-01-23 08:30:00    8,92697779920074 
2020-01-23 09:30:00    3,576198576494   
Name: avg_tot_cpu_usage_ps, Length: 697, dtype: object

In [33]:
with open("4week_transformed_droppedErrors_noFillNa.pkl", "wb") as pklfile:
    pickle.dump(result, pklfile)

In [34]:
result = result.fillna(method="ffill") # forward fill

In [35]:
result

,transactions_ps,opt_sga_size,sga_total,fixed_sga,java_pool,shared_pool,buffer_cache,pga_total,other_sga_memory,log_buffer,large_pool,streams_pool,total_pga_allocated,hardParseBindMismElTimePs,javaExecutionElapsedTimePs,plsqlEexecutionElapsedTimePs,connManageCallElapsedTimePs,failedParseElapsedTimePs,dbTimePs,hardParseSharCritElTimePs,plsqlCompilationElapsedTimePs,bgCpuTimePs,sqlExecuteElapsedTimePs,parseTimeElapsedPs,failedParseSharMemElTimePs,repeatedBindElapsedTimePs,dbCpuPs,rmanCpuTimeBackupRestorePs,inboundPlsqlRpcElapsedTimePs,sequenceLoadElapsedTimePs,hardParseElapsedTimePs,averageWaitTime,totalWaitsFG,averageWaitTimeFG,totalWaitTimeFG,totalWaits,totalWaitTime,shared_free_pct,large_free_pct,java_free_pct,session_usage,logons,user_limit,opencursors,process_usage,total_memory,genericErrors,archiveHungErrors,blockCorruptErrors,mediaFailureErrors,sessTerminateErrors,user_cpu_time_cnt,avg_user_cpu_time_pct,user_wait_time_pct,host_cpu_usage_pct,other_wait_cnt,userio_wait_cnt,avg_sync_singleblk_read_latency,sortsdisk_ps,physreads_pt,iombs_ps,recurscalls_pt,physreadslob_pt,logreads_pt,sortsdisk_pt,consistentreadgets_ps,redowrites_ps,recurscalls_ps,softparse_pct,failedparses_ps,userrollbackundorec_pt,physreadsdir_ps,sortsrows_ps,dbblkgets_ps,redosize_ps,dbtime_ps,logreads_ps,tabscanslong_ps,dbtime_pt,dbblkgets_pt,iorequests_ps,redosize_pt,executeswoparse_pct,networkbytes_ps,tabscanslong_pt,usercall_pct,rows_psort,executions_ps,usercalls_ps,physwriteslob_ps,logons_pt,opncurs_ps,physwritesdir_pt,avg_active_sessions,parses_ps,crblks_ps,dbblkchanges_ps,enqtimeouts_ps,leafnodesplits_pt,commits_ps,physwrites_pt,dbblkchanges_pt,branchnodesplits_pt,indxscanstotal_ps,parses_pt,physwrites_ps,sortsrows_pt,leafnodesplits_ps,tabscanstotal_pt,tabscanstotal_ps,sortsmemory_ps,enqreqs_pt,hardparses_ps,sortsmemory_pt,enqwaits_pt,crundorecs_ps,enqtimeouts_pt,physreadsdir_pt,crblks_pt,executions_pt,branchnodesplits_ps,enqreqs_ps,physreadslob_ps,physwriteslob_pt,consistentreadchanges_pt,commits_pt,userrollbackundorec_ps,failedparses_pt,crundorecs_pt,indxscansfull_ps,physreads_ps,indxscansfull_pt,consistentreadchanges_ps,hardparses_pt,bgcheckpoints_ps,logons_ps,dbwrcheckpoints_ps,usercalls_pt,consistentreadgets_pt,redowrites_pt,enqwaits_ps,indxscanstotal_pt,enqdeadlocks_pt,rollbacks_ps,physwritesdir_ps,rollbacks_pt,opncurs_pt,enqdeadlocks_ps,avg_users_waiting_on_class,dbtime_waitclass_pct,interconnect_rate,corrupt,currentgets_cs,lost,cr_request_cs,time_cs,libcache_miss_pct,dictionaryhit_pct,pxdwngrd75_ps,cursorcachehit_pct,redologalloc_hit_pct,cpuusage_pt,pxdwngrd25_ps,dictionarymiss_pct,response_time_pt,bufcachehit_pct,cpu_time_pct,pxdwngrdserial_ps,pgacachehit_pct,cpuusage_ps,pxdwngrdserial_pt,pxdwngrd50_ps,libcache_hit_pct,inmem_sort_pct,dumpAvail,dumpTotal,dumpUsedPercent,dumpUsed,scn_intrinsic_growth_rate,max_tot_cpu_usage_ps,avg_tot_cpu_usage_ps
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-12-25 08:45:00,"1,586",10240,"20479,998","21,732",320,4928,14336,"1144,103",384,"42,266",320,128,"1147,1",0,0,",394",0,0,",534",",001",0,",063",",531",",009",0,0,",143",0,0,0,",008",",61",140,",49",18,6522,18,"38,85358711341759","17,665987014770508",100,"29,5634920634921",111,",00000258442014515968",131,"49,6666666666667","21545,57",0,0,0,0,0,",207",",37","64,26478099258024","21,16630608177691","1,245",",011",",025",0,"2,876","1,084","942,09",0,"1891,678",0,"3021,098","9,9","1816,551","98,648",",09",0,",023","18022,693","626,461","105006,01","137,1","3647,559",0,"71,102","324,892","52,29","54457,662","52,851","816,778",0,",137",10804144,"655,464","2,494",",449",",21","641,351",",485","1,371","309,042",",007","604,81",0,",262","1,927","23,285","313,663",",002","257,805","160,274","44,898","9346,834",",506",",084",",162","14,275","28,253","4,179","7,403",",374",",002",0,",012",",003","339,933",",003","54,477",0,"

In [36]:
result.head()

,transactions_ps,opt_sga_size,sga_total,fixed_sga,java_pool,shared_pool,buffer_cache,pga_total,other_sga_memory,log_buffer,large_pool,streams_pool,total_pga_allocated,hardParseBindMismElTimePs,javaExecutionElapsedTimePs,plsqlEexecutionElapsedTimePs,connManageCallElapsedTimePs,failedParseElapsedTimePs,dbTimePs,hardParseSharCritElTimePs,plsqlCompilationElapsedTimePs,bgCpuTimePs,sqlExecuteElapsedTimePs,parseTimeElapsedPs,failedParseSharMemElTimePs,repeatedBindElapsedTimePs,dbCpuPs,rmanCpuTimeBackupRestorePs,inboundPlsqlRpcElapsedTimePs,sequenceLoadElapsedTimePs,hardParseElapsedTimePs,averageWaitTime,totalWaitsFG,averageWaitTimeFG,totalWaitTimeFG,totalWaits,totalWaitTime,shared_free_pct,large_free_pct,java_free_pct,session_usage,logons,user_limit,opencursors,process_usage,total_memory,genericErrors,archiveHungErrors,blockCorruptErrors,mediaFailureErrors,sessTerminateErrors,user_cpu_time_cnt,avg_user_cpu_time_pct,user_wait_time_pct,host_cpu_usage_pct,other_wait_cnt,userio_wait_cnt,avg_sync_singleblk_read_latency,sortsdisk_ps,physreads_pt,iombs_ps,recurscalls_pt,physreadslob_pt,logreads_pt,sortsdisk_pt,consistentreadgets_ps,redowrites_ps,recurscalls_ps,softparse_pct,failedparses_ps,userrollbackundorec_pt,physreadsdir_ps,sortsrows_ps,dbblkgets_ps,redosize_ps,dbtime_ps,logreads_ps,tabscanslong_ps,dbtime_pt,dbblkgets_pt,iorequests_ps,redosize_pt,executeswoparse_pct,networkbytes_ps,tabscanslong_pt,usercall_pct,rows_psort,executions_ps,usercalls_ps,physwriteslob_ps,logons_pt,opncurs_ps,physwritesdir_pt,avg_active_sessions,parses_ps,crblks_ps,dbblkchanges_ps,enqtimeouts_ps,leafnodesplits_pt,commits_ps,physwrites_pt,dbblkchanges_pt,branchnodesplits_pt,indxscanstotal_ps,parses_pt,physwrites_ps,sortsrows_pt,leafnodesplits_ps,tabscanstotal_pt,tabscanstotal_ps,sortsmemory_ps,enqreqs_pt,hardparses_ps,sortsmemory_pt,enqwaits_pt,crundorecs_ps,enqtimeouts_pt,physreadsdir_pt,crblks_pt,executions_pt,branchnodesplits_ps,enqreqs_ps,physreadslob_ps,physwriteslob_pt,consistentreadchanges_pt,commits_pt,userrollbackundorec_ps,failedparses_pt,crundorecs_pt,indxscansfull_ps,physreads_ps,indxscansfull_pt,consistentreadchanges_ps,hardparses_pt,bgcheckpoints_ps,logons_ps,dbwrcheckpoints_ps,usercalls_pt,consistentreadgets_pt,redowrites_pt,enqwaits_ps,indxscanstotal_pt,enqdeadlocks_pt,rollbacks_ps,physwritesdir_ps,rollbacks_pt,opncurs_pt,enqdeadlocks_ps,avg_users_waiting_on_class,dbtime_waitclass_pct,interconnect_rate,corrupt,currentgets_cs,lost,cr_request_cs,time_cs,libcache_miss_pct,dictionaryhit_pct,pxdwngrd75_ps,cursorcachehit_pct,redologalloc_hit_pct,cpuusage_pt,pxdwngrd25_ps,dictionarymiss_pct,response_time_pt,bufcachehit_pct,cpu_time_pct,pxdwngrdserial_ps,pgacachehit_pct,cpuusage_ps,pxdwngrdserial_pt,pxdwngrd50_ps,libcache_hit_pct,inmem_sort_pct,dumpAvail,dumpTotal,dumpUsedPercent,dumpUsed,scn_intrinsic_growth_rate,max_tot_cpu_usage_ps,avg_tot_cpu_usage_ps
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-12-25 08:45:00,"1,586",10240,"20479,998","21,732",320,4928,14336,"1144,103",384,"42,266",320,128,"1147,1",0,0,",394",0,0,",534",",001",0,",063",",531",",009",0,0,",143",0,0,0,",008",",61",140,",49",18,6522,18,"38,85358711341759","17,665987014770508",100,"29,5634920634921",111,",00000258442014515968",131,"49,6666666666667","21545,57",0,0,0,0,0,",207",",37","64,26478099258024","21,16630608177691","1,245",",011",",025",0,"2,876","1,084","942,09",0,"1891,678",0,"3021,098","9,9","1816,551","98,648",",09",0,",023","18022,693","626,461","105006,01","137,1","3647,559",0,"71,102","324,892","52,29","54457,662","52,851","816,778",0,",137",10804144,"655,464","2,494",",449",",21","641,351",",485","1,371","309,042",",007","604,81",0,",262","1,927","23,285","313,663",",002","257,805","160,274","44,898","9346,834",",506",",084",",162","14,275","28,253","4,179","7,403",",374",",002",0,",012",",003","339,933",",003","54,477",0,"

In [37]:
result = result.fillna(method="bfill") # backward fill

In [38]:
result.head()

,transactions_ps,opt_sga_size,sga_total,fixed_sga,java_pool,shared_pool,buffer_cache,pga_total,other_sga_memory,log_buffer,large_pool,streams_pool,total_pga_allocated,hardParseBindMismElTimePs,javaExecutionElapsedTimePs,plsqlEexecutionElapsedTimePs,connManageCallElapsedTimePs,failedParseElapsedTimePs,dbTimePs,hardParseSharCritElTimePs,plsqlCompilationElapsedTimePs,bgCpuTimePs,sqlExecuteElapsedTimePs,parseTimeElapsedPs,failedParseSharMemElTimePs,repeatedBindElapsedTimePs,dbCpuPs,rmanCpuTimeBackupRestorePs,inboundPlsqlRpcElapsedTimePs,sequenceLoadElapsedTimePs,hardParseElapsedTimePs,averageWaitTime,totalWaitsFG,averageWaitTimeFG,totalWaitTimeFG,totalWaits,totalWaitTime,shared_free_pct,large_free_pct,java_free_pct,session_usage,logons,user_limit,opencursors,process_usage,total_memory,genericErrors,archiveHungErrors,blockCorruptErrors,mediaFailureErrors,sessTerminateErrors,user_cpu_time_cnt,avg_user_cpu_time_pct,user_wait_time_pct,host_cpu_usage_pct,other_wait_cnt,userio_wait_cnt,avg_sync_singleblk_read_latency,sortsdisk_ps,physreads_pt,iombs_ps,recurscalls_pt,physreadslob_pt,logreads_pt,sortsdisk_pt,consistentreadgets_ps,redowrites_ps,recurscalls_ps,softparse_pct,failedparses_ps,userrollbackundorec_pt,physreadsdir_ps,sortsrows_ps,dbblkgets_ps,redosize_ps,dbtime_ps,logreads_ps,tabscanslong_ps,dbtime_pt,dbblkgets_pt,iorequests_ps,redosize_pt,executeswoparse_pct,networkbytes_ps,tabscanslong_pt,usercall_pct,rows_psort,executions_ps,usercalls_ps,physwriteslob_ps,logons_pt,opncurs_ps,physwritesdir_pt,avg_active_sessions,parses_ps,crblks_ps,dbblkchanges_ps,enqtimeouts_ps,leafnodesplits_pt,commits_ps,physwrites_pt,dbblkchanges_pt,branchnodesplits_pt,indxscanstotal_ps,parses_pt,physwrites_ps,sortsrows_pt,leafnodesplits_ps,tabscanstotal_pt,tabscanstotal_ps,sortsmemory_ps,enqreqs_pt,hardparses_ps,sortsmemory_pt,enqwaits_pt,crundorecs_ps,enqtimeouts_pt,physreadsdir_pt,crblks_pt,executions_pt,branchnodesplits_ps,enqreqs_ps,physreadslob_ps,physwriteslob_pt,consistentreadchanges_pt,commits_pt,userrollbackundorec_ps,failedparses_pt,crundorecs_pt,indxscansfull_ps,physreads_ps,indxscansfull_pt,consistentreadchanges_ps,hardparses_pt,bgcheckpoints_ps,logons_ps,dbwrcheckpoints_ps,usercalls_pt,consistentreadgets_pt,redowrites_pt,enqwaits_ps,indxscanstotal_pt,enqdeadlocks_pt,rollbacks_ps,physwritesdir_ps,rollbacks_pt,opncurs_pt,enqdeadlocks_ps,avg_users_waiting_on_class,dbtime_waitclass_pct,interconnect_rate,corrupt,currentgets_cs,lost,cr_request_cs,time_cs,libcache_miss_pct,dictionaryhit_pct,pxdwngrd75_ps,cursorcachehit_pct,redologalloc_hit_pct,cpuusage_pt,pxdwngrd25_ps,dictionarymiss_pct,response_time_pt,bufcachehit_pct,cpu_time_pct,pxdwngrdserial_ps,pgacachehit_pct,cpuusage_ps,pxdwngrdserial_pt,pxdwngrd50_ps,libcache_hit_pct,inmem_sort_pct,dumpAvail,dumpTotal,dumpUsedPercent,dumpUsed,scn_intrinsic_growth_rate,max_tot_cpu_usage_ps,avg_tot_cpu_usage_ps
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-12-25 08:45:00,"1,586",10240,"20479,998","21,732",320,4928,14336,"1144,103",384,"42,266",320,128,"1147,1",0,0,",394",0,0,",534",",001",0,",063",",531",",009",0,0,",143",0,0,0,",008",",61",140,",49",18,6522,18,"38,85358711341759","17,665987014770508",100,"29,5634920634921",111,",00000258442014515968",131,"49,6666666666667","21545,57",0,0,0,0,0,",207",",37","64,26478099258024","21,16630608177691","1,245",",011",",025",0,"2,876","1,084","942,09",0,"1891,678",0,"3021,098","9,9","1816,551","98,648",",09",0,",023","18022,693","626,461","105006,01","137,1","3647,559",0,"71,102","324,892","52,29","54457,662","52,851","816,778",0,",137",10804144,"655,464","2,494",",449",",21","641,351",",485","1,371","309,042",",007","604,81",0,",262","1,927","23,285","313,663",",002","257,805","160,274","44,898","9346,834",",506",",084",",162","14,275","28,253","4,179","7,403",",374",",002",0,",012",",003","339,933",",003","54,477",0,"

In [39]:
with open("4week_transformed_droppedErrors_filled.pkl", "wb") as pklfile:
    pickle.dump(result, pklfile)

In [40]:
result.shape

(2789, 181)

relevantFeatures = ["timestamp", "dbCpuPs", "dbTimePs", "bgCpuTimePs", "totalWaitTime",
                    "averageWaitTime", "process_usage", "session_usage", "user_wait_time_pct",
                    "other_sga_memory", "cpuusage_ps", "sga_total", "avg_active_sessions", "session_usage"
                   ]
res = result[relevantFeatures]

with open("multivariatUnifiedTs.pkl", "wb") as pickleFile:
     pickle.dump(result, pickleFile)

def setAllValuesToOneColumn(row):
    global df
    relevant = df[df.unifiedTimestamp == row.timestamp]
    print(relevant[allMetrics])
#         for metric in allMetrics:
#             if ((val.metric).notnull()):
#                 row.metric = val.metric

with open("multivariatUnifiedTs.pkl", "rb") as pickleFile:
    test = pickle.load(pickleFile)
    
test[relevantFeatures].sort_values(by="timestamp")

with open("allIstatXData.pkl", "rb") as pklfile:
    alternate = pickle.load(pklfile)

test.columns

test = test.dropna(axis = 1, how = "all")

test.columns

test[relevantFeatures].sort_values(by="timestamp")

endresult = test[relevantFeatures].dropna()

with open("istatRelevantFeaturesDroppedNAs.pkl", "wb") as pickleFile:
     pickle.dump(endresult, pickleFile)